In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import unidecode
%matplotlib inline

In [17]:
hilary = pd.read_csv('data/hilary_scores.csv')

In [18]:
def clean(text):
    def remove_html_tags(data):
        p = re.compile(r'<[^<]*?>')
        return p.sub('', data)

    content = remove_html_tags(text)
    return unidecode.unidecode(content).replace("\n"," ").replace("\'s","").replace("\'t","")


In [19]:
hilary['Comment'] = hilary.apply(lambda row: clean(row['Comment']), axis = 1)

/Users/datascientist/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.


In [21]:
hilary_test = hilary.to_csv('data/hilary_test.csv')

In [ ]:
for c in candidates:
    data = pd.read_csv('data/'+c+'_comments')
    data['Sentiment'] = data.apply(lambda row: sentiment(row['Comment']), axis = 1)
    data['Sentiment_b'] = data['Sentiment'] >= 0
    data['Sentiment_b'] = data['Sentiment_b'].astype(int) 
    data['Subjectivity'] = data.apply(lambda row: subj(row['Comment']), axis = 1)
    data['Subjectivity_b'] = data['Subjectivity'] >= 0
    data['Subjectivity_b'] = data['Subjectivity_b'].astype(int) 
    data.to_csv('data/'+c+'_scores.csv')

In [ ]:
candidates = ['hilary', 'sanders', 'biden', 'trump', 'bush', 'carson']

In [ ]:
hilary = pd.read_csv('data/hilary_scores.csv')
sanders = pd.read_csv('data/sanders_scores.csv')
biden = pd.read_csv('data/biden_scores.csv')

In [ ]:
from pattern.en import sentiment, polarity, subjectivity
def sentiment(text):
    return polarity(text)

def subj(text):
    return subjectivity(text)

In [ ]:
hilary = pd.read_csv('data/hilary_score.csv')
hilary['Sentiment'] = hilary.apply(lambda row: sentiment(row['Comment']), axis = 1)
hilary['Subjectivity'] = hilary.apply(lambda row: subj(row['Comment']), axis = 1)
hilary['Sentiment2'] = hilary['Comment'] 

In [ ]:
def sentiment_new(text):
    def remove_html_tags(data):
        p = re.compile(r'<[^<]*?>')
        return p.sub('', data)

    def Sentences(paragraph):
        sentenceEnders = re.compile('[.!?]')
        sentenceList = sentenceEnders.split(paragraph)
        return sentenceList

    content = remove_html_tags(text)
    content = unidecode.unidecode(content).replace("\n"," ").replace("\'", " ")
    s_max = 0
    comment_score = 0

    for i in Sentences(content):
        if abs(sentiment(i)) > abs(s_max):
            comment_score = sentiment(i)
            s_max = sentiment(i)
    return comment_score

In [ ]:
def subj_new(text):
    def remove_html_tags(data):
        p = re.compile(r'<[^<]*?>')
        return p.sub('', data)

    def Sentences(paragraph):
        sentenceEnders = re.compile('[.!?]')
        sentenceList = sentenceEnders.split(paragraph)
        return sentenceList

    content = remove_html_tags(text)
    content = unidecode.unidecode(content).replace("\n"," ").replace("\'", " ")
    s_max = 0
    comment_score = 0

    for i in Sentences(content):
        if abs(subj(i)) > abs(s_max):
            comment_score = subj(i)
            s_max = subj(i)
    return comment_score